In [20]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from tensorflow.keras.layers import Dense,Input,GlobalMaxPooling1D,LSTM,Embedding,Conv1D,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns

In [21]:
train_data=pd.read_csv('/kaggle/input/anyas-gojo-revival-space-shop/train.tsv', sep='\t')
train_data.head()

,date,time,id,reviewer name,reviewer address,contact number,variation,verified_reviews,sentiment,feedback
0,30-Jul-18,02:16:00,5138527,Theresa Diaz,"2373 Long Mews\nSouth Lorraine, FL 83715",7445448464,Black Plus,Love my new Echo Plus,5,1
1,30-Jul-18,20:13:24,7881409,Michael Wagner,"34120 James Well Apt. 930\nSouth Kenneth, AK 5...",(202)233-6662,Configuration: Fire TV Stick,Works really well.,5,1
2,29-Jul-18,03:20:48,5658396,Erin Rodriguez,"7996 Booth Mission\nTaylorport, AL 13457",(219)882-0705,Black Show,Echo Show is said to work with certain apps bu...,2,0
3,30-Jul-18,22:48:21,4317957,Christine Perez,USS Schaefer\nFPO AA 86432,291-900-6087,Heather Gray Fabric,Very cool product. Speaker sounds good with my...,5,1
4,23-Jul-18,12:36:42,7570569,Jamie Davila,"000 Kevin Cliffs Suite 674\nBrennanview, NM 77690",9527296610,Black Plus,So far I love it. It was easy to set up - if ...,5,1


In [22]:
train_data.isnull().sum()

date                0
time                0
id                  0
reviewer name       0
reviewer address    0
contact number      0
variation           0
verified_reviews    1
sentiment           0
feedback            0
dtype: int64

In [23]:
train_data.fillna({"verified_reviews":"no review"}, inplace=True)

In [24]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [25]:
from scipy.special import softmax
pov_review=[]
neg_review=[]
neu_review=[]
com_review=[]
pov_ratings=[]
neu_ratings=[]
neg_ratings=[]
for i in train_data['verified_reviews']:
    try:
        encoded_text = tokenizer(i, return_tensors='pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        neg_review.append(scores[0])
        neu_review.append(scores[1])
        pov_review.append(scores[2])
        com_review.append((scores[0]**2+scores[1]**2+scores[2]**2)**(0.5))
        if max(scores)==scores[0]:
            neg_ratings.append(1.0)
            neu_ratings.append(0.0)
            pov_ratings.append(0.0)
        elif max(scores)==scores[1]:
            neg_ratings.append(0.0)
            neu_ratings.append(1.0)
            pov_ratings.append(0.0)
        elif max(scores)==scores[2]:
            neg_ratings.append(0.0)
            neu_ratings.append(0.0)
            pov_ratings.append(1.0)
    except RuntimeError:
        print(1)
        pov_review.append(1.0)
        neg_review.append(0.0)
        neu_review.append(0.0)
        com_review.append(1.0)
        neg_ratings.append(0.0)
        neu_ratings.append(0.0)
        pov_ratings.append(1.0)

1
1


In [49]:
new_dataframe=pd.DataFrame({"positive":pov_review,"negative":neg_review,"neutral":neu_review,"combined":com_review,"feedback":list(map(lambda x: float(x),train_data['feedback'])),"isPositive":pov_ratings,"isNegative":neg_ratings,"isNeutral":neu_ratings})
new_dataframe.head()

,positive,negative,neutral,combined,feedback,isPositive,isNegative,isNeutral
0,0.989508,0.001466,0.009026,0.989550,1.0,1.0,0.0,0.0
1,0.936137,0.004737,0.059127,0.938014,1.0,1.0,0.0,0.0
2,0.107922,0.570545,0.321533,0.663741,0.0,0.0,1.0,0.0
3,0.988388,0.001345,0.010267,0.988442,1.0,1.0,0.0,0.0
4,0.991431,0.002093,0.006476,0.991454,1.0,1.0,0.0,0.0


graphs to be made:no of positive,negative and neutrals,(barchart)
graph on feedback
barchart on ratings
variation graph
confusion matrix and related graphs


In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
X_train, X_test, y_train, y_test = train_test_split(new_dataframe, train_data['sentiment'],  test_size=0.009, random_state=42) 
dtree_model = RandomForestClassifier(n_estimators=2000 ,random_state=0).fit(X_train, y_train) 
# dtree_model= CatBoostClassifier(iterations=2000, 
#                                    loss_function='MultiClass', 
#                                    random_seed=42,verbose=0).fit(X_train, y_train) 
dtree_predictions = dtree_model.score(X_test,y_test) 
print(dtree_predictions)

0.8695652173913043


In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from sklearn.preprocessing import OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
y_train = y_train.to_numpy()
encoder = OneHotEncoder(sparse=False, categories='auto')
y_train_encoded = encoder.fit_transform(y_train.reshape(-1, 1))
def build_model():
    model = Sequential([
        Dense(512, activation='relu',input_dim=5),
        Dense(256, activation='relu'),
        Dense(128,activation='relu'),
        Dense(64,activation='relu'),
        Dense(32,activation='relu'),
        Dense(16,activation='relu'),
        Dense(5, activation='softmax')  # Output layer with 'softmax' for multi-class classification
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model
estimator = KerasClassifier(build_fn = build_model, epochs = 20, batch_size = 20, verbose = 1)
sscaler = StandardScaler()
X_train = sscaler.fit_transform(X_train)
estimator.fit(X_train, y_train_encoded)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/tmp/ipykernel_32/3698744590.py:21: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn = build_model, epochs = 20, batch_size = 20, verbose = 1)


Epoch 1/20
125/125 [==============================] - 2s 5ms/step - loss: 0.2996 - accuracy: 0.7081
Epoch 2/20
125/125 [==============================] - 1s 5ms/step - loss: 0.2268 - accuracy: 0.7686
Epoch 3/20
125/125 [==============================] - 1s 5ms/step - loss: 0.2230 - accuracy: 0.7719
Epoch 4/20
125/125 [==============================] - 1s 5ms/step - loss: 0.2241 - accuracy: 0.7698
Epoch 5/20
125/125 [==============================] - 1s 5ms/step - loss: 0.2231 - accuracy: 0.7715
Epoch 6/20
125/125 [==============================] - 1s 5ms/step - loss: 0.2188 - accuracy: 0.7755
Epoch 7/20
125/125 [==============================] - 1s 5ms/step - loss: 0.2206 - accuracy: 0.7731
Epoch 8/20
125/125 [==============================] - 1s 5ms/step - loss: 0.2187 - accuracy: 0.7787
Epoch 9/20
125/125 [==============================] - 1s 5ms/step - loss: 0.2178 - accuracy: 0.7763
Epoch 10/20
125/125 [==============================] - 1s 5ms/step - loss: 0.2187 - accuracy: 0.7803

In [56]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import f1_score
y_pred = dtree_model.predict(X_test)
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(macro_f1)

0.9093333333333333


In [40]:
test_data=pd.read_csv('/kaggle/input/anyas-gojo-revival-space-shop/test.tsv', sep='\t')
pov_test_review=[]
neg_test_review=[]
neu_test_review=[]
com_test_review=[]
for i in test_data['verified_reviews']:
    try:
        encoded_text = tokenizer(i, return_tensors='pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
#         print(scores)
        neg_test_review.append(scores[0])
        neu_test_review.append(scores[1])
        pov_test_review.append(scores[2])
        com_test_review.append((scores[0]**2+scores[1]**2+scores[2]**2)**(0.5))
    except RuntimeError:
        print(1)
        pov_test_review.append(1.0)
        neg_test_review.append(0.0)
        neu_test_review.append(0.0)
        com_test_review.append(1.0)

In [41]:
test_dataframe=pd.DataFrame({"positive":pov_test_review,"negative":neg_test_review,"neutral":neu_test_review,"combined":com_test_review,"feedback":test_data['feedback']})
test_dataframe.head()

,positive,negative,neutral,combined,feedback
0,0.984899,0.002404,0.012697,0.984984,1
1,0.986175,0.003311,0.010514,0.986236,1
2,0.005175,0.963042,0.031783,0.963580,0
3,0.858581,0.032428,0.108991,0.866078,1
4,0.963731,0.003297,0.032972,0.964301,1


In [42]:
arr=dtree_model.predict(test_dataframe)
print(arr)

[5 5 1 5 5 5 5 5 1 3 5 5 5 4 5 5 1 5 5 4 5 5 5 5 5 3 2 5 5 1 5 5 5 5 5 5 5
 5 5 5 5 1 5 5 5 5 5 5 5 5 1 5 5 5 5 5 5 5 5 5 5 5 5 5 4 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 1 5 5 5 5 1 5 3 5 5 5 5 2 5 5 5 4 5 5 5 5 5 4 1 5 2 5 5 5 5 4 5
 5 1 5 5 5 5 5 5 5 5 4 5 5 3 5 5 5 5 5 5 5 5 5 5 5 5 4 5 5 5 5 5 5 5 5 5 5
 5 3 5 5 4 5 5 5 5 4 5 2 5 5 4 4 5 5 5 5 5 5 5 5 5 5 4 5 5 2 5 5 5 5 5 2 5
 2 5 5 4 5 5 5 5 5 5 5 4 5 5 5 5 5 5 4 5 5 3 1 5 5 4 5 5 5 5 4 5 4 5 5 5 5
 5 5 5 5 5 5 1 5 5 5 4 5 5 1 5 5 5 5 5 5 5 5 3 5 5 3 3 1 1 5 4 5 5 5 2 5 5
 5 4 3 5 5 5 4 5 5 5 5 3 5 5 1 5 5 5 5 3 5 4 3 3 5 5 5 5 5 4 4 4 5 5 5 5 5
 5 5 5 5 5 5 1 5 5 5 5 5 5 5 5 5 5 1 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 3 5 4 5
 5 5 5 5 5 5 5 5 5 1 5 4 5 5 5 5 5 1 5 5 5 5 4 1 5 5 4 4 4 4 5 5 4 5 5 5 4
 5 5 5 5 1 5 5 5 5 5 1 5 5 5 5 5 5 1 1 3 5 5 5 5 1 3 5 5 1 5 5 5 5 5 4 5 4
 5 5 5 5 4 5 4 5 5 5 5 2 5 3 5 4 5 5 5 5 4 5 5 5 5 5 5 5 5 5 5 5 5 4 2 3 5
 3 5 5 4 5 2 5 5 5 1 5 5 5 4 5 5 5 5 5 4 5 5 1 4 5 5 5 5 5 5 4 4 2 5 5 5 5
 5 5 5 3 5 5 5 5 4 2 4 4 

In [44]:
submission=pd.DataFrame({"id":test_data['id'],"sentiment":arr})
submission.to_csv("submission631.csv",index=False)

In [45]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [46]:
import pandas as pd
sid = SentimentIntensityAnalyzer()
train_data=pd.read_csv('/kaggle/input/anyas-gojo-revival-space-shop/train.tsv', sep='\t')
train_data.fillna({"verified_reviews":"no review"}, inplace=True)
pov_review=[]
neg_review=[]
neu_review=[]
com_review=[]
for sentence in train_data['verified_reviews']:
    sentiment_scores = sid.polarity_scores(sentence)
    pov_review.append(sentiment_scores['pos'])
    neg_review.append(sentiment_scores['neg'])
    neu_review.append(sentiment_scores['neu'])
    com_review.append(sentiment_scores['compound'])

In [47]:
new_dataframe=pd.DataFrame({"positive":pov_review,"negative":neg_review,"neutral":neu_review,"combined":com_review,"feedback":train_data['feedback']})
new_dataframe.head()

,positive,negative,neutral,combined,feedback
0,0.512,0.000,0.488,0.6369,1
1,0.545,0.000,0.455,0.3384,1
2,0.181,0.024,0.795,0.9654,0
3,0.355,0.000,0.645,0.6697,1
4,0.411,0.000,0.589,0.9550,1


In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
X_train, X_test, y_train, y_test = train_test_split(new_dataframe, train_data['sentiment'],  test_size=0.01, random_state=42) 
dtree_model = RandomForestClassifier(n_estimators=1000 ,random_state=42).fit(X_train, y_train) 
# dtree_model= CatBoostClassifier(iterations=2000, 
#                                    depth=6,        
#                                    learning_rate=0.02,  
#                                    loss_function='MultiClass', 
#                                    eval_metric='Accuracy', 
#                                    random_seed=42,verbose=0).fit(X_train, y_train) 
dtree_predictions = dtree_model.score(X_test,y_test) 
print(dtree_predictions)

0.7307692307692307
